จาก 3. เราคำนวณทุก matrix เลยซึ่งทำให้ matrix path ใหญ่มากๆ สิ่งที่จะทำในขั้นนี้คือเรายังคงทำ exact-method เหมือนเดิมโดยลดทอนขนาด matrix ให้เล็กลงจาก path ที่มี 
- task ทั้งหมด
- จุด end

เหมือนกัน

เช่น

กรณีที่เริ่มจากจุด A 

Path 1 : A-B,B-C,C-A,A-C

Path 2 : A-C,C-A,A-B,B-C 



จะเห็นว่า task และจุด end เหมือนกันดังนั้นในการคำนวณ nextNode จาก remainingTasks จะให้ผลเหมือนกัน ดังนั้นเราสามารถเลือกแค่วิธีการเดียวแล้วรันต่อไปได้ สุดท้ายถ้าทางนี้น้อยสุดจริง จะได้ผลลัพธ์ที่น้อยสุดเหมือนกัน

In [2]:
import numpy as np
import pandas as pd
import time
import multiprocessing as mp
print('Number of Core :',mp.cpu_count())

class VehicleRoutingArrangement:
    def __init__(self, node_names, distances_matrix, tasks):
        self.full_node_names = node_names
        self.node_names = node_names['name'].values
        self.distances = distances_matrix
        self.tasks = tasks
        
    def checkRemainingTasks(self, job_done):
        remaining_tasks = self.tasks.copy()
        if len(remaining_tasks) > 0:
            active_path = tasks['path'].values
            job_done = job_done[np.isin(job_done, active_path)]
            
            job_unique, job_counts = np.unique(job_done, return_counts=True)
            job_done_df = pd.DataFrame({'path': job_unique, 'Count': job_counts})
            
            merged_df = remaining_tasks.merge(job_done_df, on='path',how='outer')
            merged_df.loc[merged_df['Count_y'].isnull(), 'Count_y'] = 0
            merged_df['Count'] = merged_df['Count_x'] - merged_df['Count_y']
            output = merged_df[merged_df['Count'] > 0][['Start','End','path','Count']]        
            return output
        else:
            return remaining_tasks
    
    def startOrEnd(self, current_position,rm_task):
        start_set = rm_task['Start'].unique()
        if current_position in start_set:
            return rm_task[rm_task['Start'] == current_position]['End'].values
        else:
            return start_set
        
    def nextJobsMatrix(self, job_done, next_nodes):
        current_position = job_done[-1].split('-')[-1]
        next_jobs = current_position + '-' + next_nodes
        
        if len(job_done[0]) > 1:
            ini_mat = np.tile(job_done,(len(next_jobs),1))
            output = np.hstack((ini_mat, np.array([next_jobs]).T ))
        else:
            output = np.array([next_jobs]).T
        return output
    
    def jobMatrix(self, each_row):
        remaining_jobs = self.checkRemainingTasks(each_row)
        if len(remaining_jobs) == 0:
            job_matrix = np.append(each_row,'0')
        else:
            current_position = each_row[-1].split('-')[-1]
            next_nodes = self.startOrEnd(current_position, remaining_jobs)
            job_matrix = self.nextJobsMatrix(each_row, next_nodes)
        return job_matrix
    
    def currentPosition(self, x):
        return x.split('-')[-1]
    
    def allPath(self, before_path):
        start_time = time.time()
            
        # Pool Multiple Processing    
        pool = mp.Pool(processes=4)
        output_list = pool.map(self.jobMatrix, before_path)
        output = np.vstack(output_list)
        # re-output to unique matrix
        old_output = output.copy()
        vCurrentPos = np.vectorize(self.currentPosition)
        last_position = old_output[:,-1]
        current_pos_arr = vCurrentPos(last_position)
        output.sort(axis=1)
        all_groups = pd.DataFrame({
                        'tasks' : ["".join(i) for i in output.astype(str)], 
                        'current_pos': list(current_pos_arr) , 
                        'id': range(len(current_pos_arr)) }).groupby(['tasks','current_pos'])
        unique_index = all_groups.first()['id'].values
        output = old_output[unique_index,:]

        last_column = output[:,-1]
        all_zero = sum(last_column == '0')
        stop_time = time.time() - start_time
        print('Output Matrix Shape :', output.shape, ', Use Time :', stop_time, 's')
        if all_zero == len(last_column):
            return output
        else:
            return self.allPath(output)
    
    def getDistance(self, startEndStr):
        tasks_arr = self.tasks['path'].values
        if startEndStr == '0':
            return 0
        else:
            start,end = startEndStr.split('-')
            i,j = np.where(self.node_names == start)[0][0],np.where(self.node_names == end)[0][0]
            return self.distances[i,j] if startEndStr in tasks_arr else -1*self.distances[i,j]
        
    def sumDistance(self, distance_arr):
        all_distances = sum(distance_arr)
        pos_distances = sum(distance_arr[distance_arr > 0])
        neg_distances = sum(distance_arr[distance_arr < 0])
        abs_distances = pos_distances + abs(neg_distances)
        return pos_distances, neg_distances, all_distances, abs_distances
        
    def allDistancePath(self, all_path):
        getDistanceAllElement = np.vectorize(self.getDistance) 
        result_array = getDistanceAllElement(all_path)
        all_distance_path = np.apply_along_axis(self.sumDistance, 1, result_array)
        return all_distance_path
    
    def getTaskName(self, task_path):
        name_arr = self.full_node_names['name'].values
        only_task = task_path[task_path != '0'];

        # แยกจุดเริ่ม-จบ 
        start_f = lambda x : x.split('-')[0]
        start_vf = np.vectorize(start_f)
        start_node = start_vf(only_task)
        end_f = lambda y : y.split('-')[1]
        end_vf = np.vectorize(end_f)
        end_node = end_vf(only_task)

        row_num = len(only_task)

        name_list = []
        distance_list = []
        for i in range(row_num):
            # name        
            start_name =  self.full_node_names[self.full_node_names['name'] == start_node[i]]['node'].values[0]
            end_name =  self.full_node_names[self.full_node_names['name'] == end_node[i]]['node'].values[0]
            name = start_name + '-' + end_name 
            name_list.append(name)

            # distance        
            start,end = start_node[i],end_node[i]
            m,n = np.where(name_arr == start)[0][0],np.where(name_arr == end)[0][0]
            before_path = only_task[0:i]
            remaining_tasks = self.checkRemainingTasks(before_path)
            current_task = task_path[i] 
            if current_task in remaining_tasks['path'].values:
                distance_list.append(self.distances[m,n])
            else:
                distance_list.append(-self.distances[m,n])

        return pd.DataFrame({'Name' : name_list, 'Distance': distance_list})
    
start_point = np.array([['S']]) # start_point
node_names = pd.read_csv('data/node_names.csv')
distances = pd.read_csv('data/distances_matrix.csv', header=None).values
tasks = pd.read_csv('data/demo_tasks.csv')[['Start','End','Count']]
tasks['path'] = tasks['Start'] +'-'+ tasks['End']

Number of Core : 4


In [3]:
vehicleObj = VehicleRoutingArrangement(node_names, distances, tasks)

In [4]:
all_path = vehicleObj.allPath(start_point)

Output Matrix Shape : (1, 1) , Use Time : 0.08915996551513672 s
Output Matrix Shape : (2, 2) , Use Time : 0.0850071907043457 s
Output Matrix Shape : (4, 3) , Use Time : 0.08620190620422363 s
Output Matrix Shape : (8, 4) , Use Time : 0.09043192863464355 s
Output Matrix Shape : (15, 5) , Use Time : 0.10927987098693848 s
Output Matrix Shape : (24, 6) , Use Time : 0.14958524703979492 s
Output Matrix Shape : (32, 7) , Use Time : 0.1804060935974121 s
Output Matrix Shape : (37, 8) , Use Time : 0.20919418334960938 s
Output Matrix Shape : (35, 9) , Use Time : 0.20987415313720703 s
Output Matrix Shape : (31, 10) , Use Time : 0.24719786643981934 s
Output Matrix Shape : (22, 11) , Use Time : 0.24539422988891602 s
Output Matrix Shape : (14, 12) , Use Time : 0.22520804405212402 s
Output Matrix Shape : (14, 13) , Use Time : 0.16047096252441406 s
Output Matrix Shape : (14, 14) , Use Time : 0.21397185325622559 s


In [5]:
np.set_printoptions(threshold=np.nan,linewidth=200)
all_path

array([['S-T', 'T-B', 'B-N', 'N-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-P', 'P-B', 'B-P', '0', '0'],
       ['S-T', 'T-A', 'A-B', 'B-N', 'N-A', 'A-P', 'P-A', 'A-P', 'P-B', 'B-P', 'P-B', 'B-P', '0', '0'],
       ['S-T', 'T-B', 'B-P', 'P-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-N', 'N-B', 'B-P', '0', '0'],
       ['S-T', 'T-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-N', 'N-B', 'B-P', 'P-B', 'B-P', '0', '0'],
       ['S-T', 'T-B', 'B-P', 'P-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-P', 'P-B', 'B-N', '0', '0'],
       ['S-T', 'T-A', 'A-P', 'P-A', 'A-P', 'P-A', 'A-B', 'B-P', 'P-B', 'B-P', 'P-B', 'B-N', '0', '0'],
       ['S-T', 'T-B', 'B-P', 'P-B', 'B-P', 'P-B', 'B-N', 'N-A', 'A-B', 'B-A', 'A-P', 'P-A', 'A-P', '0'],
       ['S-T', 'T-A', 'A-P', 'P-B', 'B-P', 'P-B', 'B-P', 'P-B', 'B-N', 'N-A', 'A-B', 'B-A', 'A-P', '0'],
       ['S-T', 'T-B', 'B-N', 'N-B', 'B-P', 'P-B', 'B-P', 'P-A', 'A-B', 'B-A', 'A-P', 'P-A', 'A-P', '0'],
       ['S-T', 'T-A', 'A-P', 'P-B', 'B-N', 'N-B', 'B-P', 'P-B', 'B-

In [6]:
# np.load('../script/output/all_path.npy').shape

(360, 14)

In [6]:
distance_path = vehicleObj.allDistancePath(all_path) # unique อยู่แล้ว
np.sort(distance_path[:,2])

array([312.3, 330.2, 331.3, 338.1, 341.8, 349.2, 355.5, 356. , 357.1, 359.7, 360.8, 373.4, 375. , 378.7])

In [8]:
path_from_number_of_distances = all_path[distance_path[:,2] == 378.7] # เอาค่าดีสุด
path_from_number_of_distances

array([['S-T', 'T-A', 'A-B', 'B-N', 'N-A', 'A-P', 'P-A', 'A-P', 'P-B', 'B-P', 'P-B', 'B-P', '0', '0']], dtype=object)

In [9]:
thai_name = vehicleObj.getTaskName(path_from_number_of_distances[0])
thai_name

,Name,Distance
0,ลำปลายมาศ-วังแดง,322.0
1,วังแดง-นครหลวง,-11.6
2,นครหลวง-ท่าเรือ,19.8
3,ท่าเรือ-หนองแค(สัตว์บก),43.2
4,หนองแค(สัตว์บก)-นครหลวง,-40.3
5,นครหลวง-โคกตูม,45.6
6,โคกตูม-นครหลวง,-45.6
7,นครหลวง-โคกตูม,45.6
8,โคกตูม-ท่าเรือ,-29.5
9,ท่าเรือ-โคกตูม,29.5
